# Grounding in LTN (cont.)

In [1]:
import ltn
import numpy as np
import tensorflow as tf

In [2]:
# Connectives

"""
LTN suppports various logical connectives. 
They are grounded using fuzzy semantics.
"""

Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or  = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())

In [ ]:
# The wrapper ltn.Wrapper_Connective allows to use 
# the operators with LTN formulas. 
# It takes care of combining sub-formulas 
# that have different variables appearing in them (the sub-formulas may have different dimensions that need to be "broadcasted").

In [3]:
x = ltn.Variable('x',np.random.normal(0.,1.,(10,2))) # 10 values 
y = ltn.Variable('y',np.random.normal(0.,2.,(5,2))) # 5 values 

c1 = ltn.Constant([0.5,0.0], trainable=False)
c2 = ltn.Constant([4.0,2.0], trainable=False)

Eq = ltn.Predicate.Lambda(lambda args: tf.exp(-tf.norm(args[0]-args[1],axis=1))) # predicate measuring similarity

Eq([c1,c2])

ltn.Formula(tensor=0.01775427535176277, free_vars=[])

In [4]:
Not(Eq([c1,c2]))


ltn.Formula(tensor=0.9822457432746887, free_vars=[])

In [5]:
Implies(Eq([c1,c2]), Eq([c2,c1]))


ltn.Formula(tensor=0.9824644327163696, free_vars=[])

In [ ]:
# Notice the dimension of the outcome: the result is evaluated for every x. 
And(Eq([x,c1]), Eq([x,c2]))


ltn.Formula(tensor=[0.00214702 0.00474421 0.00261408 0.00476047 0.00582067 0.00154182
 0.00689567 0.01763404 0.00062498 0.00258841], free_vars=['x'])

In [7]:

# Notice the dimensions of the outcome: the result is evaluated for every x and y.
# Notice also that y did not appear in the 1st argument of `Or`; 
# the connective broadcasts the results of its two arguments to match.
Or(Eq([x,c1]), Eq([x,y]))

ltn.Formula(tensor=[[0.3622932  0.35747913 0.3775126  0.35409173 0.37542492]
 [0.2793247  0.22513804 0.20591693 0.19862263 0.27885318]
 [0.28614837 0.29084194 0.26069286 0.2515632  0.29441223]
 [0.42177668 0.40450934 0.39781895 0.3856792  0.43446332]
 [0.43106443 0.40357855 0.3971988  0.38573346 0.4441353 ]
 [0.29597288 0.28007978 0.39106914 0.3065094  0.31441626]
 [0.6040121  0.578861   0.6189379  0.5830253  0.62585676]
 [0.5519495  0.26955435 0.2825934  0.26628715 0.6208621 ]
 [0.19880602 0.20880814 0.23084672 0.21393465 0.20789489]
 [0.35100827 0.35190696 0.34365067 0.32995054 0.3623639 ]], free_vars=['x', 'y'])

In [8]:
# Quantifiers

# existential quantification ("exists"): the generalized mean (pMean)
# universal quantification ("for all"): the generalized mean of "the deviations w.r.t. the truth" (pMeanError)

Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=5),semantics="exists")

In [9]:
x = ltn.Variable('x',np.random.normal(0.,1.,(10,2))) # 10 values in R²
y = ltn.Variable('y',np.random.normal(0.,2.,(5,2))) # 5 values in R²

Eq = ltn.Predicate.Lambda(lambda args: tf.exp(-tf.norm(args[0]-args[1],axis=1))) # predicate measuring similarity

Eq([x,y])

ltn.Formula(tensor=[[0.01652003 0.01916156 0.30587107 0.03787563 0.7952408 ]
 [0.06203888 0.04719335 0.09592509 0.09088384 0.18674628]
 [0.10642341 0.0669755  0.04048703 0.09496512 0.07570548]
 [0.01475791 0.00356557 0.4834509  0.05403107 0.19655758]
 [0.06547122 0.04248425 0.10195924 0.09997451 0.1915149 ]
 [0.07105985 0.06270643 0.06560171 0.0846884  0.13005114]
 [0.0156939  0.00243857 0.28378087 0.0619686  0.11933742]
 [0.03722332 0.10599384 0.05379205 0.04198394 0.1253895 ]
 [0.03038935 0.01766533 0.319089   0.07377867 0.5271765 ]
 [0.00976038 0.00172176 0.24724007 0.03880996 0.0965015 ]], free_vars=['x', 'y'])

In [10]:
Forall(x,Eq([x,y]))


ltn.Formula(tensor=[0.0424481  0.03643149 0.18728364 0.06760752 0.2129637 ], free_vars=['y'])

In [11]:
Forall((x,y),Eq([x,y]))


ltn.Formula(tensor=0.10617011785507202, free_vars=[])

In [12]:
Exists((x,y),Eq([x,y]))


ltn.Formula(tensor=0.3797701895236969, free_vars=[])

In [13]:
Forall(x, Exists(y, Eq([x,y])))


ltn.Formula(tensor=0.21139734983444214, free_vars=[])

#### pMean can be understood as a smooth-maximum that depends on the hyper-paramer :

- p -> 1: the operator tends to mean,
- p -> inf : the operator tends to max.

#### Similarly, pMeanError can be understood as asmooth-minimum:

- p -> 1  : the operator tends to mean,
- p -> inf: the operator tends to min.

#### Therefore, 
 #### p offers flexibility in writing more or less strict formulas, to account for outliers in the data depending on the application. Note that this can have strong implications for training (see complementary notebook). One can set a default value for 
 #### when initializing the operator, or can use different values at each call of the operator.

In [ ]:
#Diagonal Quantification
# The values are generated at random, for the sake of illustration.
# In a real scenario, they would come from a dataset.
samples = np.random.rand(100,2,2) # 100 R^{2x2} values 
labels = np.random.randint(3, size=100) # 100 labels (class 0/1/2) that correspond to each sample 
onehot_labels = tf.one_hot(labels,depth=3)

x = ltn.Variable("x",samples) 
l = ltn.Variable("l",onehot_labels)

class ModelC(tf.keras.Model):
    def __init__(self):
        super(ModelC, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    def call(self, inputs):
        x, l = inputs[0], inputs[1]
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return tf.math.reduce_sum(x*l,axis=1)

C = ltn.Predicate(ModelC())

TypeError: 'Variable' object is not iterable